# Comparing Pipeline Runs with Vertex AI Experiments


## Learning objectives

1. Formalize a training component.
2. Build a training pipeline.
3. Run several Pipeline jobs and log their results.
4. Compare different Pipeline jobs.

## Introduction
In this notebook, you learn how to use `Vertex AI Experiments` to log a pipeline job and compare different pipeline jobs.

Depending on the model life cycle of your data science team, you would like to experiment and track training Pipeline runs and its associated parameters. Then, you would to compare runs of these Pipelines to each others in order to figure out which is the best configuration generates the model you will register in the Vertex AI Model Registry.

### Dataset

The dataset used for this tutorial is the [Iris dataset](https://www.tensorflow.org/datasets/catalog/iris) from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview). This dataset does not require any feature engineering. The version of the dataset you will use in this tutorial is stored in a public Cloud Storage bucket. The trained model predicts the type of Iris flower species from a class of three species: setosa, virginica, or versicolor.

Each learning objective will correspond to a __#TODO__ in the [student lab notebook](../labs/comparing_pipeline_runs.ipynb) -- try to complete that notebook first before reviewing this solution notebook. 


### Install additional packages

Install additional package dependencies not installed in your notebook environment, such as Vertex AI SDK and KFP SDK. Use the latest major GA version of each package.

In [1]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

!pip3 install {USER_FLAG} --force-reinstall 'google-cloud-aiplatform>=1.15' -q --no-warn-conflicts
!pip3 install {USER_FLAG} kfp -q --no-warn-conflicts

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [2]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set your project ID

In [1]:
PROJECT_ID = "[your-project-ID]"  # Replace with your Project ID

In [2]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [3]:
REGION = "[your-region]"  # Replace with your region

if REGION == "[your-region]":
    REGION = "us-central1"

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.


In [4]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [5]:
BUCKET_NAME = "[your-bucket-name]"  # Replace with your unique bucket name
BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [6]:
# Create a regional Cloud Storage bucket
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Creating gs://qwiklabs-gcp-00-1fe7a1835a07/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [7]:
! gsutil ls -al $BUCKET_URI

#### Service Account

**If you don't know your service account**, try to get your service account using `gcloud` command by executing the second cell below.

In [8]:
SERVICE_ACCOUNT = "[your-service-account]"  # Replace '[your-service-account]' with your project's Compute Engine service account

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### Set service account access for Vertex AI Pipelines

Run the following commands to grant your service account access to read and write pipeline artifacts in the bucket that you created in the previous step -- you only need to run these once per service account.

In [9]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### Download training dataset


In [10]:
DATASET_URI = "gs://cloud-samples-data/ai-platform/iris"

!gsutil cp -r $DATASET_URI $BUCKET_URI

Copying gs://cloud-samples-data/ai-platform/iris/classification/evaluate.csv [Content-Type=text/csv]...
Copying gs://cloud-samples-data/ai-platform/iris/classification/train.csv [Content-Type=text/csv]...
Copying gs://cloud-samples-data/ai-platform/iris/iris_data.csv [Content-Type=application/octet-stream]...
Copying gs://cloud-samples-data/ai-platform/iris/iris_predict.csv [Content-Type=text/csv]...
- [4 files][  5.4 KiB/  5.4 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://cloud-samples-data/ai-platform/iris/iris_target.csv [Content-Type=application/octet-stream]...
Copying gs://cloud-samples-data/ai-platform/iris/iris_test.csv [Content-Type=text/csv]...
Copying gs://cloud-samples-data/ai-platform/iris/iris_training.cs

### Import libraries and define constants

In [11]:
import logging
# General
import os
import time

logger = logging.getLogger("logger")
logging.basicConfig(level=logging.INFO)

import kfp.v2.compiler as compiler
# Pipeline Experiments
import kfp.v2.dsl as dsl
# Vertex AI
from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform_v1.types.pipeline_state import PipelineState
from kfp.v2.dsl import Metrics, Model, Output, component

In [12]:
# Experiments
TASK = "classification"
MODEL_TYPE = "xgboost"
EXPERIMENT_NAME = f"{PROJECT_ID}-{TASK}-{MODEL_TYPE}-{UUID}"

# Pipeline
PIPELINE_TEMPLATE_FILE = "pipeline.json"
PIPELINE_URI = f"{BUCKET_URI}/pipelines"
TRAIN_URI = f"{BUCKET_URI}/iris/iris_data.csv"
LABEL_URI = f"{BUCKET_URI}/iris/iris_target.csv"
MODEL_URI = f"{BUCKET_URI}/model"

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [13]:
vertex_ai.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

### Set pre-built containers

Set the pre-built Docker container image for training and prediction.


For the latest list, see [Pre-built containers for training](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers).


For the latest list, see [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers).

In [14]:
TRAIN_IMAGE = vertex_ai.helpers.get_prebuilt_prediction_container_uri(
    framework="xgboost", framework_version="1.1", accelerator="cpu"
)

## Formalize your training as pipeline component


Before you start running your pipeline experiments, you have to formalize your training as pipeline component.

To do that, you build the pipeline by using the `kfp.v2.dsl.component` decorator to convert your training task into a pipeline component. 

In [15]:
@component(
    packages_to_install=[
        "numpy==1.18.5",
        "pandas==1.0.4",
        "scikit-learn==0.23.1",
        "xgboost==1.1.1",
    ]
)
def custom_trainer(
    train_uri: str,
    label_uri: str,
    max_depth: int,
    learning_rate: float,
    boost_rounds: int,
    model_uri: str,
    metrics: Output[Metrics],
    model_metadata: Output[Model],
):

    # import libraries
    import logging
    import uuid
    from pathlib import Path as path

    import pandas as pd
    import xgboost as xgb
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split

    # variables
    gs_prefix = "gs://"
    gcsfuse_prefix = "/gcs/"
    train_path = train_uri.replace(gs_prefix, gcsfuse_prefix)
    label_path = label_uri.replace(gs_prefix, gcsfuse_prefix)
    model_path = model_uri.replace(gs_prefix, gcsfuse_prefix)

    def get_logger():
        """
        Get the logger
        """
        logger = logging.getLogger(__name__)
        logger.setLevel(logging.INFO)
        handler = logging.StreamHandler()
        handler.setFormatter(
            logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
        )
        logger.addHandler(handler)
        return logger

    def get_data(
        train_path: str, label_path: str
    ) -> (xgb.DMatrix, pd.DataFrame, pd.DataFrame):
        """
        Get the data
        Args:
            train_path: the path of the train data
            label_path: the path of the label data
        Returns:
            the train data and the label data
        """
        # Load data into pandas, then use `.values` to get NumPy arrays
        data = pd.read_csv(train_path).values
        labels = pd.read_csv(label_path).values

        # Convert one-column 2D array into 1D array for use with XGBoost
        labels = labels.reshape((labels.size,))
        train_data, test_data, train_labels, test_labels = train_test_split(
            data, labels, test_size=0.2, random_state=7
        )

        # Load data into DMatrix object
        dtrain = xgb.DMatrix(train_data, label=train_labels)
        return dtrain, test_data, test_labels

    def train_model(max_depth: int, eta: int, boost_rounds, dtrain: xgb.DMatrix):
        """
        Train the model
        Args:
            max_depth: the max depth of the model
            eta: the eta of the model
            boost_rounds: the boost rounds of the model
            dtrain: the train data
        Returns:
            the trained model
        """
        # Train XGBoost model
        param = {"max_depth": max_depth, "eta": eta}
        model = xgb.train(param, dtrain, num_boost_round=boost_rounds)
        return model

    def evaluate_model(model, test_data, test_labels):
        """
        Evaluate the model
        Args:
            model: the trained model
            test_data: the test data
            test_labels: the test labels
        Returns:
            the accuracy of the model
        """
        dtest = xgb.DMatrix(test_data)
        pred = model.predict(dtest)
        predictions = [round(value) for value in pred]
        # Evaluate predictions
        accuracy = accuracy_score(test_labels, predictions)
        return accuracy

    def save_model(model, model_path):
        """
        Save the model
        Args:
            model: the trained model
            model_path: the path of the model
        """
        model_id = str(uuid.uuid1())
        model_path = f"{model_path}/{model_id}/model.bst"
        path(model_path).parent.mkdir(parents=True, exist_ok=True)
        model.save_model(model_path)

    # Main ----------------------------------------------

    dtrain, test_data, test_labels = get_data(train_path, label_path)
    model = train_model(max_depth, learning_rate, boost_rounds, dtrain)
    accuracy = evaluate_model(model, test_data, test_labels)
    save_model(model, model_path)

    # Metadata ------------------------------------------
    metrics.log_metric("accurancy", accuracy)
    model_metadata.uri = model_uri

## Build a pipeline

Below code will perform creating pipelineJob in associated project.

In [16]:
@dsl.pipeline(name="custom-training-pipeline")
def pipeline(
    train_uri: str,
    label_uri: str,
    max_depth: int,
    learning_rate: float,
    boost_rounds: int,
    model_uri: str,
):

    custom_trainer(
        train_uri, label_uri, max_depth, learning_rate, boost_rounds, model_uri
    )

### Compile your pipeline into a JSON file

In [17]:
# Compile the pipeline with required parameters
compiler.Compiler().compile(pipeline_func=pipeline, package_path="pipeline.json")

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1293: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


## Submit and track pipeline runs

### Submit Pipeline runs

Now that you have the pipeline, you define its training configuration depending on the defined parameters. Below you have an example and how you can submit several pipeline runs. 

In [18]:
runs = [
    {"max_depth": 4, "learning_rate": 0.2, "boost_rounds": 10},
    {"max_depth": 5, "learning_rate": 0.3, "boost_rounds": 20},
    {"max_depth": 3, "learning_rate": 0.1, "boost_rounds": 30},
    {"max_depth": 6, "learning_rate": 0.5, "boost_rounds": 40},
    {"max_depth": 5, "learning_rate": 0.4, "boost_rounds": 30},
]

In [19]:
for i, run in enumerate(runs):

    job = vertex_ai.PipelineJob(
        display_name=f"{EXPERIMENT_NAME}-pipeline-run-{i}",
        template_path=PIPELINE_TEMPLATE_FILE,
        pipeline_root=PIPELINE_URI,
        parameter_values={
            "train_uri": TRAIN_URI,
            "label_uri": LABEL_URI,
            "model_uri": MODEL_URI,
            **run,
        },
    )
    # Submit the pipeline job using job.submit()
    job.submit(experiment=EXPERIMENT_NAME)

Creating PipelineJob


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


PipelineJob created. Resource name: projects/21417957250/locations/us-central1/pipelineJobs/custom-training-pipeline-20221014105059


INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/21417957250/locations/us-central1/pipelineJobs/custom-training-pipeline-20221014105059


To use this PipelineJob in another session:


...

INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/21417957250/locations/us-central1/pipelineJobs/custom-training-pipeline-20221014105111')


View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20221014105111?project=21417957250


INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20221014105111?project=21417957250


Associating projects/21417957250/locations/us-central1/pipelineJobs/custom-training-pipeline-20221014105111 to Experiment: qwiklabs-gcp-00-1fe7a1835a07-classification-xgboost-406gb4wq


INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/21417957250/locations/us-central1/pipelineJobs/custom-training-pipeline-20221014105111 to Experiment: qwiklabs-gcp-00-1fe7a1835a07-classification-xgboost-406gb4wq


### Check Pipeline run states

Vertex AI SDK provides you `get_experiment_df` method to monitor the status of pipeline runs. You can use it either to return parameters and metrics of the Pipeline Runs in the Vertex AI Experiment or in combination with `get` method of `PipelineJob` to return the pipeline job in Vertex AI Pipeline.


In [20]:
# see state of all pipelineJob
vertex_ai.get_experiment_df(EXPERIMENT_NAME)

,experiment_name,run_name,run_type,state,param.max_depth,param.boost_rounds,param.learning_rate,param.label_uri,param.model_uri,param.train_uri
0,qwiklabs-gcp-00-1fe7a1835a07-classification-xg...,custom-training-pipeline-20221014105111,system.PipelineRun,RUNNING,5,30,0.4,gs://qwiklabs-gcp-00-1fe7a1835a07/iris/iris_ta...,gs://qwiklabs-gcp-00-1fe7a1835a07/model,gs://qwiklabs-gcp-00-1fe7a1835a07/iris/iris_da...
1,qwiklabs-gcp-00-1fe7a1835a07-classification-xg...,custom-training-pipeline-20221014105108,system.PipelineRun,RUNNING,6,40,0.5,gs://qwiklabs-gcp-00-1fe7a1835a07/iris/iris_ta...,gs://qwiklabs-gcp-00-1fe7a1835a07/model,gs://qwiklabs-gcp-00-1fe7a1835a07/iris/iris_da...
2,qwiklabs-gcp-00-1fe7a1835a07-classification-xg...,custom-training-pipeline-20221014105105,system.PipelineRun,RUNNING,3,30,0.1,gs://qwiklabs-gcp-00-1fe7a1835a07/iris/iris_ta...,gs://qwiklabs-gcp-00-1fe7a1835a07/model,gs://qwiklabs-gcp-00-1fe7a1835a07/iris/iris_da...
3,qwiklabs-gcp-00-1fe7a1835a07-classification-xg...,custom-training-pipeline-20221014105102,system.PipelineRun,RUNNING,5,20,0.3,gs://qwiklabs-gcp-00-1fe7a1835a07/iris/iris_ta...,gs://qwiklabs-gcp-00-1fe7a1835a07/model,gs://qwiklabs-gcp-00-1fe7a1835a07/iris/iris_da...
4,qwiklabs-gcp-00-1fe7a1835a07-classification-xg...,custom-training-pipeline-20221014105059,system.PipelineRun,RUNNING,4,10,0.2,gs://qwiklabs-gcp-00-1fe7a1835a07/iris/iris_ta...,gs://qwiklabs-gcp-00-1fe7a1835a07/model,gs://qwiklabs-gcp-00-1fe7a1835a07/iris/iris_da...


The pipeline runs in the Vertex AI Experiment will be monitored based on pipeline run status.

In [21]:
while True:
    pipeline_experiments_df = vertex_ai.get_experiment_df(EXPERIMENT_NAME)
    if any(
        pipeline_state != "COMPLETE" for pipeline_state in pipeline_experiments_df.state
    ):
        print("Pipeline runs are still running...")
        if any(
            pipeline_state == "FAILED"
            for pipeline_state in pipeline_experiments_df.state
        ):
            print("At least one Pipeline run failed")
            break
    else:
        print("Pipeline experiment runs have completed")
        break
    time.sleep(60)

Pipeline runs are still running...
Pipeline runs are still running...
Pipeline experiment runs have completed


In [22]:
# Get the PipelineJob resource using the experiment run name
pipeline_experiments_df = vertex_ai.get_experiment_df(EXPERIMENT_NAME)
job = vertex_ai.PipelineJob.get(pipeline_experiments_df.run_name[0])
print("Pipeline job name: ", job.resource_name)
print("Pipeline Run UI link: ", job._dashboard_uri())

Pipeline job name:  projects/21417957250/locations/us-central1/pipelineJobs/custom-training-pipeline-20221014105111
Pipeline Run UI link:  https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-training-pipeline-20221014105111?project=21417957250


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial

In [23]:
# Delete the pipeline
while True:
    for i in range(0, len(runs)):
        pipeline_job = vertex_ai.PipelineJob.get(pipeline_experiments_df.run_name[i])
        if pipeline_job.state != PipelineState.PIPELINE_STATE_SUCCEEDED:
            print("Pipeline job is still running...")
            time.sleep(60)
        else:
            print("Pipeline job is complete.")
            pipeline_job.delete()
    break

# Delete experiment
exp = vertex_ai.Experiment(EXPERIMENT_NAME)
exp.delete()

# Delete bucket
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}

# Remove local files

!rm {PIPELINE_TEMPLATE_FILE}

Pipeline job is complete.
Deleting PipelineJob : projects/21417957250/locations/us-central1/pipelineJobs/custom-training-pipeline-20221014105111


INFO:google.cloud.aiplatform.base:Deleting PipelineJob : projects/21417957250/locations/us-central1/pipelineJobs/custom-training-pipeline-20221014105111


Delete PipelineJob  backing LRO: projects/21417957250/locations/us-central1/operations/5165028782842576896


INFO:google.cloud.aiplatform.base:Delete PipelineJob  backing LRO: projects/21417957250/locations/us-central1/operations/5165028782842576896


PipelineJob deleted. . Resource name: projects/21417957250/locations/us-central1/pipelineJobs/custom-training-pipeline-20221014105111


...

INFO:google.cloud.aiplatform.base:Deleting Context : projects/21417957250/locations/us-central1/metadataStores/default/contexts/qwiklabs-gcp-00-1fe7a1835a07-classification-xgboost-406gb4wq


Delete Context  backing LRO: projects/21417957250/locations/us-central1/metadataStores/default/contexts/qwiklabs-gcp-00-1fe7a1835a07-classification-xgboost-406gb4wq/operations/3271828089486704640


INFO:google.cloud.aiplatform.base:Delete Context  backing LRO: projects/21417957250/locations/us-central1/metadataStores/default/contexts/qwiklabs-gcp-00-1fe7a1835a07-classification-xgboost-406gb4wq/operations/3271828089486704640


Context deleted. . Resource name: projects/21417957250/locations/us-central1/metadataStores/default/contexts/qwiklabs-gcp-00-1fe7a1835a07-classification-xgboost-406gb4wq


INFO:google.cloud.aiplatform.base:Context deleted. . Resource name: projects/21417957250/locations/us-central1/metadataStores/default/contexts/qwiklabs-gcp-00-1fe7a1835a07-classification-xgboost-406gb4wq
